In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df = pd.read_json('../output/badm004/badminton004_clip_00000197/alphapose-results.json')

In [ ]:
df.head()


In [ ]:
df.keypoints[0]

In [ ]:
df[['image_id','score']].head()

In [ ]:
coco_format ={   0:"Nose",
    1:"LEye",
    2:"REye",
    3:"LEar",
    4:"REar",
    5:"LShoulder",
    6:  "RShoulder",
    7:  "LElbow",
    8:  "RElbow",
    9:  "LWrist",
    10: "RWrist",
    11: "LHip",
    12: "RHip",
    13: "LKnee",
    14: "Rknee",
    15: "LAnkle",
    16: "RAnkle"}

In [ ]:
coco_format_flip = {v:k for k,v in coco_format.items()}

In [ ]:
def get_3_tuples(keypoints):
    tuples=[]
    for i in range(int(len(keypoints)/3)):
        tuples.append((keypoints[3*i],keypoints[3*i+1],keypoints[3*i+2]))
    return tuples

In [ ]:
get_3_tuples(df.keypoints[0])

In [ ]:
def get_width(keypoints):
    tuples = get_3_tuples(keypoints)
    xs = [i[0] for i in tuples]
    return np.max(xs) - np.min(xs)

In [ ]:
def get_height(keypoints):
    tuples = get_3_tuples(keypoints)
    ys = [i[1] for i in tuples]
    return np.max(ys) - np.min(ys)

In [ ]:
get_width(df.keypoints[0])

In [ ]:
get_height(df.keypoints[0])

In [ ]:
def normalize_tuples(keypoints):
    tuples = get_3_tuples(keypoints)
    xs = [i[0] for i in tuples]
    ys = [i[1] for i in tuples]
    centre = ((np.max(xs)+np.min(xs))/2,(np.max(ys)+np.min(ys))/2)
    for i in range(len(tuples)):
        tuples[i]=(tuples[i][0]-centre[0],tuples[i][1]-centre[1],tuples[i][2])

    return tuples

In [ ]:
def normalize_keypoints(keypoints):
    tuples = get_3_tuples(keypoints)
    xs = [i[0] for i in tuples]
    ys = [i[1] for i in tuples]
    centre = ((np.max(xs)+np.min(xs))/2,(np.max(ys)+np.min(ys))/2)
    for i in range(len(tuples)):
        tuples[i]=(tuples[i][0]-centre[0],tuples[i][1]-centre[1],tuples[i][2])

    return [i for tuple in tuples for i in tuple]

In [ ]:
normalize_tuples(df.keypoints[0])

In [ ]:
a=get_3_tuples(df.keypoints[0])

In [ ]:
df['normalized_tuples'] = df.keypoints.apply(normalize_tuples)

In [ ]:
df.head()

In [ ]:
type(df.normalized_tuples[0])

In [ ]:
type(df.normalized_tuples[0][1])

In [ ]:
df['width'] = df.keypoints.apply(get_width)

In [ ]:
df['height'] = df.keypoints.apply(get_height)

In [ ]:
def get_part_position(bodypart, keypoints, normalize=True):
    tuples = get_3_tuples(keypoints)
    if normalize:
        tuples = normalize_tuples(keypoints)
    try:
        return tuples[coco_format_flip[bodypart]][0:2]
    except:
        print('Something wrong')

In [ ]:
get_part_position('Nose',df.keypoints[0])

In [ ]:
get_part_position('LAnkle',df.keypoints[0])

In [ ]:
import math
math.atan2(-0.1,-1)

In [ ]:
v1= tuple(np.subtract(get_part_position('Nose',df.keypoints[0]),get_part_position('LHip',df.keypoints[0])))

In [ ]:
v2= tuple(np.subtract(get_part_position('LHip',df.keypoints[0]),get_part_position('LAnkle',df.keypoints[0])))

In [ ]:
v1,v2

In [ ]:
def get_angle_vector(v1,v2):
    x1=v1[0]
    y1=v1[1]
    x2=v2[0]
    y2=v2[1]
    angle = 180*(math.atan2(y1,x1)-math.atan2(y2,x2))/3.1417
    return abs(angle)

In [ ]:
get_angle_vector(v1,v2)

In [ ]:
def get_angle_at_hip(keypoints):
    nose = get_part_position('Nose',keypoints)
    LHip = get_part_position('LHip',keypoints)
    LAnkle = get_part_position('LAnkle',keypoints)
    RHip = get_part_position('RHip',keypoints)
    RAnkle = get_part_position('RAnkle',keypoints)
                               
    v1= tuple(np.subtract(nose,LHip))
    v2= tuple(np.subtract(LHip,LAnkle))
    left_angle = get_angle_vector(v1,v2)
                               
    v1= tuple(np.subtract(nose,RHip))
    v2= tuple(np.subtract(RHip,RAnkle))
    right_angle = get_angle_vector(v1,v2)
    return left_angle+right_angle
    

In [ ]:
def is_person_standing_straight(keypoints):
    if get_angle_at_hip(keypoints)<60:
        return 1
    else:
        return 0

In [ ]:
is_person_standing_straight(df.keypoints[0])

In [ ]:
df['is_person_standing'] = df.keypoints.apply(is_person_standing_straight)

In [ ]:
df['angle_at_hip'] = df.keypoints.apply(get_angle_at_hip)

In [ ]:
df.head()

In [ ]:
df.groupby('is_person_standing').count()

In [ ]:
df.groupby('is_person_standing').width.mean()

In [ ]:
df[df.is_person_standing==1]

In [ ]:
df[df.is_person_standing==1].normalized_tuples[7]

In [ ]:
df[df.image_id=='0.jpg']

In [ ]:
df.shape

In [ ]:
df_concat=pd.DataFrame()
for i in os.listdir('../output/badm004/'):
    path = '../output/badm004/'+i+'/alphapose-results.json'
    df = pd.read_json(path)
    df['clip']=str(i)
    df['normalized_tuples'] = df.keypoints.apply(normalize_tuples)
    df['normalized_keypoints'] = df.keypoints.apply(normalize_keypoints)
    df['width'] = df.keypoints.apply(get_width)
    df['height'] = df.keypoints.apply(get_height)
    df['angle_at_hip'] = df.keypoints.apply(get_angle_at_hip)
    df['is_person_standing'] = df.keypoints.apply(is_person_standing_straight)
    df_concat = df_concat.append(df)
    print(df_concat.shape)

In [ ]:
df_concat.groupby(['clip']).is_person_standing.mean()

In [ ]:
df_concat[df_concat.height>30].groupby(['clip']).is_person_standing.sum()

In [ ]:
df_concat[df_concat.height>30].groupby(['clip','image_id']).is_person_standing.count()

In [ ]:
#df_concat = df_concat[df_concat.height>40]

In [ ]:
res = pd.read_json('../output/badminton004.json')

In [ ]:
play_clips = [i.split('\\')[-1].split('.')[0] for i in res.video]

In [ ]:
play_clips

In [ ]:
df_concat['play'] = [1 if i in play_clips else 0 for i in df_concat['clip']  ]

In [ ]:
df_concat.groupby('play').image_id.count()

In [ ]:
df_concat.groupby('play').is_person_standing.mean()

In [ ]:
df_concat.groupby('play').angle_at_hip.mean()

In [ ]:
df_concat.groupby(['clip','image_id']).is_person_standing.sum()

In [ ]:
df_concat= df_concat.sort_values(by=['image_id', 'height'],ascending=False).drop_duplicates(['clip','image_id'])

In [ ]:
df_concat.shape

In [ ]:
df_concat

In [ ]:
df_concat.groupby('play').angle_at_hip.mean()

In [ ]:
df_concat.sort_values(by=['clip', 'image_id'])

In [ ]:
X_mat = np.vstack( (df_concat.normalized_keypoints))

In [ ]:
X_mat.shape

In [ ]:
y= df_concat.play


In [ ]:
len(y)

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=10, test_size=0.2, random_state=13)
data_split = gss.split(X_mat, y, groups = df_concat['clip'])
train_ids, test_ids = next(data_split)

In [ ]:
len(train_ids)

In [ ]:
train_ids

In [ ]:
len(test_ids)

In [ ]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_mat, y)
X_train, X_test, y_train, y_test = X_mat[train_ids],X_mat[test_ids],np.array(y)[train_ids],np.array(y)[test_ids]                                                                                  

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rfc

In [ ]:
clf = rfc()
clf.fit(X_train,y_train)

In [ ]:
pred = clf.predict(X_test)
pred_proba = clf.predict_proba(X_test)
pred_proba_1 = pred_proba[:,1]

In [ ]:
len(pred_proba_1)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

In [ ]:
sum([pred[i]==list(y_test)[i] for i in range(len(pred))])

In [ ]:
len(pred)

In [ ]:
pred

In [ ]:
df_concat=pd.DataFrame()
for i in os.listdir('../output/badm001/'):
    try:
        print(i)
        path = '../output/badm001/'+i+'/alphapose-results.json'
        df = pd.read_json(path)
        df['clip']=str(i)
        df['normalized_tuples'] = df.keypoints.apply(normalize_tuples)
        df['normalized_keypoints'] = df.keypoints.apply(normalize_keypoints)
        df['width'] = df.keypoints.apply(get_width)
        df['height'] = df.keypoints.apply(get_height)
        df['angle_at_hip'] = df.keypoints.apply(get_angle_at_hip)
        df['is_person_standing'] = df.keypoints.apply(is_person_standing_straight)
        df_concat = df_concat.append(df)
        print(df_concat.shape)
    except:
        pass

In [ ]:
df_concat

In [ ]:
df_concat= df_concat.sort_values(by=['image_id', 'height'],ascending=False).drop_duplicates(['clip','image_id'])

In [ ]:
X_mat = np.vstack( (df_concat.normalized_keypoints))

In [ ]:
pred = clf.predict(X_mat)
pred

In [ ]:
df_concat['pred']= pred

In [ ]:
len(pred)

In [ ]:
res = df_concat.groupby(['clip']).pred.mean()
res

In [ ]:
list(res.index[res>0.1])

In [ ]:
act = pd.read_json('../output/badminton001.json')

In [ ]:
act_play_clips = [i.split('\\')[-1].split('.')[0].split('_')[-1] for i in act.video]

In [ ]:
act_play_clips

In [ ]:
sum([1 if i in act_play_clips else 0 for i in [i.split('_')[-1] for i in res.index[res>0.05]]])

In [ ]:
res.index[res>0.05].shape

In [ ]:
sum([1 if i in act_play_clips else 0 for i in [i.split('_')[-1] for i in res.index]])

In [ ]:
res.index.shape

In [ ]:
res.sort_values()

In [ ]:
res.sort_values(ascending=False)

In [ ]:
f= open("clipslist.txt","w+")

for i in list(res.index[res>0.1]):
    print(i)
    try:
        f.write("file "+"../output/clips_badminton001/"+i+".mp4\n")
    except:
        pass
f.close()

#os.system('ffmpeg -f concat -safe 0 -i clipslist.txt -c copy highlights.mp4')

#os.system('rm clipslist.txt')


In [1]:
!ffmpeg -f concat -safe 0 -i clipslist.txt -c copy highlights.mp4

ffmpeg version 2.8.15-0ubuntu0.16.04.1 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 5.4.0 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 20160609
  configuration: --prefix=/usr --extra-version=0ubuntu0.16.04.1 --build-suffix=-ffmpeg --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --cc=cc --cxx=g++ --enable-gpl --enable-shared --disable-stripping --disable-decoder=libopenjpeg --disable-decoder=libschroedinger --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librtmp --enable-libschroedinger --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --

In [2]:
res

NameError: name 'res' is not defined